In [28]:
%load_ext autoreload
%autoreload 2

from mt.utils import *
import pyclesperanto as cle
import numpy as np
import napari
from tqdm import tqdm
cle.select_device("RTX")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(OpenCL) NVIDIA GeForce RTX 4070 SUPER (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              560.70
	Device Type:                 GPU
	Compute Units:               56
	Global Memory Size:          12281 MB
	Maximum Object Size:         3070 MB
	Max Clock Frequency:         2505 MHz
	Image Support:               Yes

In [18]:
path = "../../04_uCT/AD51/"
scan = load_scan(path, logging=True)


Loading images from:  ../../04_uCT/AD51/Slices/
Loaded stack with shape (1780, 322, 2267) and a size of 2.60 GB in 2.33 s.


In [19]:
def divide_scan(scan, size_gb: float = 1):
    """Returns the indexes that split an array into parts of roughly equal size less than size_gb GB."""
    n_parts = int(np.ceil(scan.nbytes / size_gb / 1e9))
    n_slices, _, _ = scan.shape
    part_size = n_slices // n_parts
    rest = n_slices % n_parts
    part_sizes = [0] + [part_size] * n_parts
    part_sizes[-1] += rest
    part_sizes = np.cumsum(part_sizes)
    return part_sizes

split_idxs = divide_scan(scan.scan)
    

In [20]:
# process_subset_in_napari(scan.scan)

In [21]:
print("size of subscan: {:.2f} MB".format(scan[:800].nbytes/1e6))

size of subscan: 1167.96 MB


In [22]:
mask = np.zeros_like(scan.scan)
for ii in tqdm(range(len(split_idxs)-1)):
    left, right = split_idxs[ii], split_idxs[ii+1]
    mask[left:right, :, :] = segment_scan(scan[left:right],
                            logging=True,
                            otsu_sigma=0.6,
                            particle_mask_sigma=0.1,
                            particle_enlarge_radius=1,
                            smooth_labels_radius=2)


  0%|          | 0/3 [00:00<?, ?it/s]

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.


 33%|███▎      | 1/3 [00:18<00:37, 18.73s/it]

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.


 67%|██████▋   | 2/3 [00:36<00:18, 18.27s/it]

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.


100%|██████████| 3/3 [00:56<00:00, 18.74s/it]


In [ ]:
save_mask(mask,
          path=path + "segmented2/")

In [25]:
viewer = napari.Viewer()
viewer.add_image(scan.scan)
viewer.add_labels(mask)

<Labels layer 'mask' at 0x21582712c50>

In [24]:
# viewer = napari.Viewer()
# viewer.add_image(subscan)
# viewer.window.add_dock_widget(segment, area='right')